# Find hospitals closest to an incident

The `network` module of the ArcGIS API for Python can be used to solve different types of network analysis operations. In this sample, we see how to find the hospital that is closest to an incident.

## Closest facility

The closest facility solver provides functionality for finding out the closest locations to a particular input point. This solver would be useful in cases when you have an incident and need to find the closest facility or need to get information on the travel time and the distance to each of the facilities from an incident point for reporting purposes.

![](http://desktop.arcgis.com/en/arcmap/latest/extensions/network-analyst/GUID-96C273DB-6A24-4D42-AADA-975A33B44F3D-web.png)

When finding closest facilities, you can specify how many to find and whether the direction of travel is toward or away from them. The closest facility solver displays the best routes between incidents and facilities, reports their travel costs, and returns driving directions.

### Connect to your GIS
As a first step, you would need to establish a connection to your organization which could be an ArcGIS Online organization or an ArcGIS Enterprise.

In [1]:
from IPython.display import HTML
import pandas as pd
from arcgis.gis import GIS

#connect to your GIS
my_gis = GIS('https://localgovne.maps.arcgis.com', 'patrick.hammons.tugis')

Enter password: ········


### Create a Network Layer
To perform any network analysis (such as finding the closest facility, the best route between multiple stops, or service area around a facility), you would need to create a `NetworkLayer` object. In this sample, since we are solving for closest facilities, we need to create a `ClosestFacilityLayer` which is a type of `NetworkLayer`.

To create any `NetworkLayer` object, you would need to provide the URL to the appropriate network analysis service. Hence, in this sample, we provide a `ClosestFacility` URL to create a `ClosestFacilityLayer` object. 

Since all ArcGIS Online organizations already have access to those routing services, you can access this URL through the `GIS` object's `helperServices` property. If you have your own ArcGIS Server based map service with network analysis capability enabled, you would need to provide the URL for this service.

Let us start by importing the `network` module

In [2]:
import arcgis.network as network

Access the analysis URL from the `GIS` object

In [3]:
analysis_url = my_gis.properties.helperServices.closestFacility.url
analysis_url

'https://route.arcgis.com/arcgis/rest/services/World/ClosestFacility/NAServer/ClosestFacility_World'

Create a `ClosestFacilityLayer` object using this URL

In [4]:
cf_layer = network.ClosestFacilityLayer(analysis_url, gis=my_gis)

### Create hospitals layer
In this sample, we will be looking for the closest hospital (facility) to an incident location. Even though we are interested in finding out the closest one, it would still be helpful to get the information on the distance and travel time to all of them for reference purposes.

In the code below, we need to geocode the hospitals' addresses as well as do the reverse geocode for the incident location which has been supplied in the latitude/longitude format.

To perform the geocode operations, we import the `geocoding` module of the ArcGIS API.

In [5]:
from arcgis import geocoding

In this sample, we geocode addresses of hospitals to create the facility layer. In your workflows, this could any feature layer. Create a list of hospitals in Rio de Janeiro, Brazil.

In [6]:
hospitals_addresses = ['Estrada Adhemar Bebiano, 339 Del Castilho, Rio de Janeiro RJ, 21051-370, Brazil',
                       'R. José dos Reis Engenho de Dentro, Rio de Janeiro RJ, 20750-000, Brazil',
                       'R. Dezessete, s/n Maré, Rio de Janeiro RJ, 21042-010, Brazil',
                       'Rua Dr. Miguel Vieira Ferreira, 266 Ramos, Rio de Janeiro RJ, Brazil']

Loop through each address and geocode it. The geocode operation returns a list of matches for each address. We pick the first result and extract the coordinates from it and construct a `Feature` object out of it. Then we combine all the `Feature`s representing the hospitals into a `FeatureSet` object.

In [7]:
from arcgis.features import Feature, FeatureSet

In [8]:
hosp_feat_list = []

for address in hospitals_addresses:
    hit = geocoding.geocode(address)[0]
    hosp_feat = Feature(geometry=hit['location'], attributes=hit['attributes'])

    hosp_feat_list.append(hosp_feat)

In [9]:
hosp_feat_list

[{"geometry": {"x": -43.26597372422451, "y": -22.874822503604097}, "attributes": {"Loc_name": "World", "Status": "M", "Score": 98.52, "Match_addr": "Estrada Adhemar Bebiano 339, Del Castilho, Rio de Janeiro, 21051-070", "LongLabel": "Estrada Adhemar Bebiano 339, Del Castilho, Rio de Janeiro, 21051-070, BRA", "ShortLabel": "Estrada Adhemar Bebiano 339", "Addr_type": "StreetAddressExt", "Type": "", "PlaceName": "", "Place_addr": "Estrada Adhemar Bebiano 339, Del Castilho, Rio de Janeiro, 21051-070", "Phone": "", "URL": "", "Rank": 20, "AddBldg": "", "AddNum": "339", "AddNumFrom": "257", "AddNumTo": "257", "AddRange": "257-257", "Side": "L", "StPreDir": "", "StPreType": "Estrada", "StName": "Adhemar Bebiano", "StType": "", "StDir": "", "BldgType": "", "BldgName": "", "LevelType": "", "LevelName": "", "UnitType": "", "UnitName": "", "SubAddr": "", "StAddr": "Estrada Adhemar Bebiano 339", "Block": "", "Sector": "", "Nbrhd": "", "District": "Del Castilho", "City": "Rio de Janeiro", "MetroAre

Construct a `FeatureSet` using each hospital `Feature`.

In [10]:
hospitals_fset = FeatureSet(features=hosp_feat_list, 
                            geometry_type='esriGeometryPoint', 
                            spatial_reference={'latestWkid': 4326})

In [11]:
hospitals_fset

<FeatureSet> 4 features

Lets draw our hospitals on a map

In [12]:
map1 = my_gis.map('Rio de Janeiro, Brazil')
map1

In [13]:
map1.draw(hospitals_fset, symbol={"type": "esriSMS","style": "esriSMSSquare",
                                  "color": [76,115,0,255],"size": 8,})

### Create incidents layer
Similarly, let us create the incient layer

In [14]:
incident_coords = '-43.281206,-22.865676'
reverse_geocode = geocoding.reverse_geocode({"x": incident_coords.split(',')[0], 
                                              "y": incident_coords.split(',')[1]})

incident_feature = Feature(geometry=reverse_geocode['location'], 
                           attributes=reverse_geocode['address'])

In [15]:
incident_fset = FeatureSet([incident_feature], geometry_type='esriGeometryPoint',
                          spatial_reference={'latestWkid': 4326})

Let us add the incident to the map

In [16]:
map1.draw(incident_fset, symbol={"type": "esriSMS","style": "esriSMSCircle","size": 8})

## Solve for closest hospital
By default the closest facility service would return only the closest location, so we need to specify explicitly the `default_target_facility_count` parameter as well as `return_facilities`.


In [17]:
result = cf_layer.solve_closest_facility(incidents=incident_fset,
                                        facilities=hospitals_fset,
                                        default_target_facility_count=4,
                                        return_facilities=True,
                                        impedance_attribute_name='TravelTime',
                                        accumulate_attribute_names=['Kilometers','TravelTime'])

Let us inspect the result dictionary

In [18]:
result.keys()

dict_keys(['messages', 'routes', 'facilities'])

Let us use the `routes` dictionary to construct line features out of the routes to display on the map

In [19]:
result['routes'].keys()

dict_keys(['fieldAliases', 'geometryType', 'spatialReference', 'features'])

In [20]:
result['routes']['features'][0].keys()

dict_keys(['attributes', 'geometry'])

Construct line features out of the routes that are returned.

In [20]:
line_feat_list = []
for line_dict in result['routes']['features']:
    f1 = Feature(line_dict['geometry'], line_dict['attributes'])
    line_feat_list.append(f1)

In [21]:
line_feat_list

[{"geometry": {"paths": [[[-43.28124189499994, -22.865707432999955], [-43.28052999999994, -22.86651999999998], [-43.28114999999997, -22.867489999999975], [-43.28123999999997, -22.868509999999958], [-43.28147999999993, -22.86880999999994], [-43.28165999999993, -22.86898999999994], [-43.28177999999997, -22.869039999999927], [-43.281879999999944, -22.869119999999953], [-43.282839999999965, -22.86985999999996], [-43.28200999999996, -22.871239999999943], [-43.28243999999995, -22.871489999999937], [-43.282959999999946, -22.87185999999997], [-43.28203999999994, -22.87302999999997], [-43.28190999999998, -22.873219999999947], [-43.28180999999995, -22.873369999999966], [-43.281709999999975, -22.873309999999947], [-43.28072999999995, -22.872819999999933], [-43.28060999999997, -22.872739999999965], [-43.280039999999985, -22.87334999999996], [-43.279249999999934, -22.87349999999998], [-43.278959999999984, -22.873549999999966], [-43.27860999999996, -22.873719999999935], [-43.277619999999956, -22.874

In [22]:
routes_fset = FeatureSet(line_feat_list, 
                         geometry_type=result['routes']['geometryType'],
                         spatial_reference= result['routes']['spatialReference'])

Add the routes back to the map. The route to the closest hospital is in red

In [23]:
map1.draw(routes_fset)

## Analyze the results in a table
Since we parsed the routes as a `FeatureSet`, we can display the attributes easily as a `pandas` `DataFrame`.

In [24]:
routes_fset.df

,FacilityCurbApproach,FacilityID,FacilityRank,IncidentCurbApproach,IncidentID,Name,ObjectID,Shape_Length,Total_Kilometers,Total_Miles,Total_TravelTime,SHAPE
0,1,1,1,2,1,Rua Tangapeme 36 - Location 1,1,0.028461,3.016378,1.874290,7.664818,"{'paths': [[[-43.28124189499994, -22.865707432..."
1,2,4,2,2,1,Rua Tangapeme 36 - Location 4,2,0.048831,5.224941,3.246628,12.579875,"{'paths': [[[-43.28124189499994, -22.865707432..."
2,1,3,3,2,1,Rua Tangapeme 36 - Location 3,3,0.066833,7.075700,4.396636,14.348580,"{'paths': [[[-43.28124189499994, -22.865707432..."
3,2,2,4,2,1,Rua Tangapeme 36 - Location 2,4,0.069660,7.350178,4.567189,14.734714,"{'paths': [[[-43.28124189499994, -22.865707432..."


Let us add the hospital addresses and incident address to this table and display only the relevant columns

In [25]:
df1 = routes_fset.df
df1['facility_address'] = hospitals_addresses
df1['incident_address'] = [incident_feature.attributes['Match_addr'] for i in range(len(hospitals_addresses))]

In [26]:
df1[['facility_address','incident_address','Total_Miles','Total_TravelTime']]


,facility_address,incident_address,Total_Miles,Total_TravelTime
0,"Estrada Adhemar Bebiano, 339 Del Castilho, Rio...","Rua Tangapeme 2-50, Complexo do Alemão, Rio de...",1.874290,7.664818
1,"R. José dos Reis Engenho de Dentro, Rio de Jan...","Rua Tangapeme 2-50, Complexo do Alemão, Rio de...",3.246628,12.579875
2,"R. Dezessete, s/n Maré, Rio de Janeiro RJ, 210...","Rua Tangapeme 2-50, Complexo do Alemão, Rio de...",4.396636,14.348580
3,"Rua Dr. Miguel Vieira Ferreira, 266 Ramos, Rio...","Rua Tangapeme 2-50, Complexo do Alemão, Rio de...",4.567189,14.734714


### Conclusion
Thus using the `network` module of the ArcGIS API for Python, you can solve for closest facilities from an incident location.